In [1]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET

In [2]:
x1 = pd.read_xml('./busstop_output.xml')

In [3]:
tree = ET.parse('busstop_output.xml')
root = tree.getroot()

In [5]:
df = pd.DataFrame()
cnt = 0
for stop in root.iter('stops'):
    for stpinf in stop.findall('stopinfo'):
        # print(stpinf.attrib)
        df.loc[cnt, 'id'] = stpinf.attrib['id']
        df.loc[cnt, 'type'] = stpinf.attrib['type']
        try:df.loc[cnt, 'busStop'] = stpinf.attrib['busStop']
        except:pass
        df.loc[cnt, 'lane'] = stpinf.attrib['lane']
        df.loc[cnt, 'pos'] = stpinf.attrib['pos']
        df.loc[cnt, 'parking'] = stpinf.attrib['parking']
        df.loc[cnt, 'started'] = stpinf.attrib['started']
        df.loc[cnt, 'ended'] = stpinf.attrib['ended']
        df.loc[cnt, 'delay'] = stpinf.attrib['delay']
        try:df.loc[cnt, 'arrivalDelay'] = stpinf.attrib['arrivalDelay']
        except:pass
        df.loc[cnt, 'initialPersons'] = stpinf.attrib['initialPersons']
        df.loc[cnt, 'loadedPersons'] = stpinf.attrib['loadedPersons']
        df.loc[cnt, 'unloadedPersons'] = stpinf.attrib['unloadedPersons']
        df.loc[cnt, 'initialContainers'] = stpinf.attrib['initialContainers']
        df.loc[cnt, 'loadedContainers'] = stpinf.attrib['loadedContainers']
        df.loc[cnt, 'unloadedContainers'] = stpinf.attrib['unloadedContainers']
        cnt+=1

In [7]:
df.busStop = df.busStop.astype(str)
df.busStop = df.busStop.apply(lambda x: x.split('-')[0])
df = df.sort_values(by=['id', 'started'])
df.started = df.started.apply(lambda x: int(float(x)))

In [8]:
def time_conv(x):
    h, m, s = x.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [105]:
gtfs_data = pd.read_csv('./trips_vid_time_stops_GTFS_20220111.csv')
gtfs_data = gtfs_data.dropna()
gtfs_data.first_stop_id = gtfs_data.first_stop_id.astype(int).astype(str)
gtfs_data.last_stop_id = gtfs_data.last_stop_id.astype(int).astype(str)
gtfs_data.vid = gtfs_data.vid.astype(str)

In [106]:
gtfs_data.gtfs_start_time = gtfs_data.gtfs_start_time.apply(lambda x: time_conv(x))
gtfs_data.gtfs_end_time = gtfs_data.gtfs_end_time.apply(lambda x: time_conv(x))

In [135]:
df = df.sort_values(['id', 'started'])
gtfs_data = gtfs_data.sort_values(['vid', 'gtfs_start_time'])

In [144]:
#based on start time
df2 = df.sort_values(['id', 'started']).merge(gtfs_data.sort_values(['vid', 'gtfs_start_time']), left_on = ['id', 'busStop'], right_on = ['vid', 'first_stop_id'], how='right')
df2['start_diff'] = abs(df2.gtfs_start_time - df2.started)
df2 = df2.sort_values(['id', 'start_diff', 'started']).reset_index(drop=True)
# df2 = df2.dropna()

In [198]:
df2_merged = df2.groupby(['trip_id']).first().reset_index().dropna().sort_values('started')
df2_merged.trip_id = df2_merged.trip_id.astype(int).astype(str) 
df2_merged.block_id = df2_merged.block_id.astype(int).astype(str) 

In [208]:
df_w_trips = df.merge(df2_merged[['trip_id', 'route_id', 'block_id', 'id', 'started']], on=['id', 'started'], how='left')
# .dropna(subset=['trip_id'])

In [209]:
import numpy as np
trpid = ''
routeid = ''
blockid = ''
for idx, val in df_w_trips.iterrows():
    if val.trip_id is not np.nan:
        trpid = val.trip_id
        routeid = val.route_id
        blockid = val.block_id
    if val.trip_id is np.nan:
        # print(val.trip_id)
        df_w_trips.loc[idx, 'trip_id'] = trpid
        df_w_trips.loc[idx, 'route_id'] = routeid
        df_w_trips.loc[idx, 'block_id'] = blockid
    

In [211]:
df_w_trips.to_csv('trips_from_blocks.csv')

In [212]:
# df.merge(gtfs_data, left_on=['id', 'busStop'], right_on=['vid', 'first_stop_id'], how='left').dropna()

In [116]:
#exporting bus stop info with initial person load
df2.groupby(['trip_id']).first().reset_index().dropna().sort_values('started').to_csv('busstop_info_new.csv', index=False)
# .merge(df, on=['id', 'started'], how='left').sort_values('started')

In [54]:
#choosing the shortest delay between gtfs start time and sim start time as the corresponding trip
df2.groupby(['trip_id']).first().reset_index().to_csv('trips_from_xml_blocks.csv', index=False)

In [13]:
#based on end time
df3 = df[['id', 'type', 'busStop', 'started']].merge(gtfs_data.sort_values(['vid', 'gtfs_end_time']), left_on = ['id', 'busStop'], right_on = ['vid', 'first_stop_id'], how='right')
df3['end_diff'] = abs(df3.gtfs_end_time - df3.started)
df3 = df3.sort_values(['id', 'end_diff', 'started']).reset_index(drop=True)